### 해리스 코너 검출
* 참고: https://dacon.io/codeshare/5106

해리스 코너 검출(Harris corner detection)은 소벨(Sobel) 미분으로 엣지를 검출하면서 엣지의 경사도 변화량을 측정하여 변화량이 X축과 Y축 모든 방향으로 크게 변화하는 것을 코너로 판단

* dst = cv2.cornerHarris(src, blockSize, ksize, k[, dst, borderType]
    + src : 입력 영상, 그레이 스케일
    + blockSize : 이웃 픽셀 범위
    + ksize : 소벨 미분 커널 크기
    + k : 코너 검출 상수, 경험적 상수(0.04~0.06)
    + dst : 코너 검출 결과
        - src와 같은 크기와 1채널 배열, 변화량의 값, 지역 최대 값이 코너점을 의미
    + borderType : 외곽 영역 보정 형식

In [1]:
import cv2
import numpy as np

img = cv2.imread('./images/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [2]:
# 해리스 코너 검출 ---①
corner = cv2.cornerHarris(gray, 2, 3, 0.04)

In [3]:
# 변화량 결과의 최대값 10% 이상의 좌표 구하기 ---②
coord = np.where(corner > 0.1* corner.max())
coord = np.stack((coord[1], coord[0]), axis=-1)

In [4]:
# 코너 좌표에 동그리미 그리기 ---③
for x, y in coord:
    cv2.circle(img, (x,y), 5, (0,0,255), 1, cv2.LINE_AA)

In [5]:
# 변화량을 영상으로 표현하기 위해서 0~255로 정규화 ---④
corner_norm = cv2.normalize(corner, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

In [6]:
# 화면에 출력
corner_norm = cv2.cvtColor(corner_norm, cv2.COLOR_GRAY2BGR)
merged = np.hstack((corner_norm, img))
cv2.imshow('Harris Corner', merged)
cv2.waitKey()
cv2.destroyAllWindows()

[참고] 시(Shi)와 토마시(Tomasi)는 해리스 코너 검출을 개선한 알고리즘을 발표했다. 이 방법으로 검출한 코너는 객체 추적에 좋은 특징이 된다고 해서 OpenCV는 cv2.goodFeaturesToTrack()이라는 이름의 함수를 제공한다.

* corners = cv2.goodFeaturesToTrack(img, maxCorners, qualityLevel, minDistance[, corners, mask, blockSize, useHarrisDetector, k])
    + img : 입력 영상
    + maxCorners : 얻고 싶은 코너 개수, 강한 것 순
    + qualityLevel : 코너로 판단할 스레시홀드 값
    + minDistance : 코너 간 최소 거리
    + mask : 검출에 제외할 마스크
    + blockSize=3 : 코너 주변 영역의 크기
    + useHarrisDetector=False : 코너 검출 방법 선택
        - True = 해리스 코너 검출 방법, False = 시와 토마시 검출 방법
    + k : 해리스 코너 검출 방법에 사용할 k 계수
    + corners : 코너 검출 좌표 결과, N x 1 x 2 크기의 배열, 실수 값 (정수 변형 필요)

In [7]:
import cv2
import numpy as np

img = cv2.imread('./images/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 시-토마스의 코너 검출 메서드
corners = cv2.goodFeaturesToTrack(gray, 80, 0.01, 10)
# 실수 좌표를 정수 좌표로 변환
corners = np.int32(corners)

# 좌표에 동그라미 표시
for corner in corners:
    x, y = corner[0]
    cv2.circle(img, (x, y), 5, (0,0,255), 1, cv2.LINE_AA)

cv2.imshow('Corners', img)
cv2.waitKey()
cv2.destroyAllWindows()